In [154]:
import numpy as np

# TODO: Information Gain function
def Information_Gain(S, branches):
    # S: float
    # branches: List[List[int]] num_branches * num_cls
    # return: float
    # raise NotImplementedError
    branches = np.array(branches)
    sum_per_attr_val = np.sum(branches, axis=1)
    probs_per_attr_val = sum_per_attr_val / np.sum(sum_per_attr_val)
    probs_per_class_in_each_attrval =  branches / sum_per_attr_val.reshape((-1,1))
    entropy_per_attrval = np.array([ np.sum([ -x * np.log2(x) if x > 0 else 0 for x in prob ])
                        for prob in probs_per_class_in_each_attrval])
    entropy = np.sum(probs_per_attr_val * entropy_per_attrval)
    return S - entropy


# TODO: implement reduced error prunning function, pruning your tree on this function
def reduced_error_prunning(decisionTree, X_test, y_test):
    # decisionTree
    # X_test: List[List[any]]
    # y_test: List
    # raise NotImplementedError
    
    # children里面可能会有None，先按没用None处理    
    def helper(decisionTree, TreeNode, X_test, y_test):
        # recursion terminate condition
        # lead node
        root = TreeNode
        if not root.splittable:
            return
    
        for child in root.children:
            if child.splittable:
                helper(decisionTree, child, X_test, y_test)
                    
        # process non-leaf node
        predicted = np.array(decisionTree.predict(X_test))
        total = len(y_test)
        y_test = np.array(y_test)
        indexes = np.where(predicted == y_test)[0]
        accuracy_pre = indexes.size / total
        # prune
        root.splittable = False
        predicted = np.array(decisionTree.predict(X_test))
        indexes = np.where(predicted == y_test)[0]
        accuracy_post = indexes.size / total
        # if accuracy after pruning less than before pruning, reset root.splirable
        if accuracy_post < accuracy_pre:
            root.splittable = True
                             
        return
    
    root = decisionTree.root_node
    helper(decisionTree, root, X_test, y_test)
    return
                             
# print current tree
def print_tree(decisionTree, node=None, name='branch 0', indent='', deep=0):
    if node is None:
        node = decisionTree.root_node
    print(name + '{')

    print(indent + '\tdeep: ' + str(deep))
    string = ''
    label_uniq = np.unique(node.labels).tolist()
    for label in label_uniq:
        string += str(node.labels.count(label)) + ' : '
    print(indent + '\tnum of samples for each class: ' + string[:-2])

    if node.splittable:
        print(indent + '\tsplit by dim {:d}'.format(node.dim_split))
        for idx_child, child in enumerate(node.children):
            print_tree(decisionTree, node=child, name='\t' + name + '->' + str(idx_child), indent=indent + '\t', deep=deep+1)
    else:
        print(indent + '\tclass:', node.cls_max)
    print(indent + '}')


In [229]:
import numpy as np


class DecisionTree():
    def __init__(self):
        self.clf_name = "DecisionTree"
        self.root_node = None

    def train(self, features, labels):
        # features: List[List[float]], labels: List[int]
        # init
        assert (len(features) > 0)
        num_cls = np.unique(labels).size

        # build the tree
        self.root_node = TreeNode(features, labels, num_cls)
        if self.root_node.splittable:
            self.root_node.split()

        return

    def predict(self, features):
        # features: List[List[any]]
        # return List[int]
        y_pred = []
        for idx, feature in enumerate(features):
            pred = self.root_node.predict(feature)
            y_pred.append(pred)
        return y_pred


class TreeNode(object):
    def __init__(self, features, labels, num_cls):
        # features: List[List[any]], labels: List[int], num_cls: int
        self.features = features
        self.labels = labels
        self.children = []
        self.num_cls = num_cls
        # find the most common labels in current node
        count_max = 0
        for label in np.unique(labels):
            if self.labels.count(label) > count_max:
                count_max = labels.count(label)
                self.cls_max = label
                # splitable is false when all features belongs to one class
        if len(np.unique(labels)) < 2:
            self.splittable = False
        else:
            self.splittable = True

        self.dim_split = None  # the index of the feature to be split

        self.feature_uniq_split = None  # the possible unique values of the feature to be split

    #TODO: try to split current node
    def split(self):
        # raise NotImplementedError
        if self.splittable == False:
            return
        
#         if all( not x  for x in self.feature_uniq_split):
#             self.splittable = False
#             return
        
        features = self.features
        features = np.array(features)
        labels = self.labels
        labels = np.array(labels)
        
        IG_list = []  # store entropy of each split beased on feature
        IG_max = -float('inf')
        for i in range(len(self.features[0])):
            # only get ith column -- one feature
            i_dim = features[:, i]
            unique_vals = np.unique(i_dim)
            
            unique_labels = np.unique(labels)
            branches = np.zeros((len(unique_vals), self.num_cls))
            for cls in range(0, self.num_cls):
                for uval in range(0, len(unique_vals)):
                    branches[uval, cls] = np.intersect1d(np.where(i_dim == unique_vals[uval]), np.where(labels == unique_labels[cls])).size
           
            # create branches used for calulate information gain
#             branches = np.zeros((len(unique_vals), labels.max() + 1))  # 如果label是文本又不能用了
#             branches = np.zeros((len(unique_vals), self.num_cls)) # 如果不连续，会越界

#             for col, attr_val in enumerate(unique_vals):
#                 y_labels = np.array(labels)[np.where(i_dim == attr_val)]
#                 # labels correspond to 0, 1, 2, 3 ...
#                 # 未必连续，不能这么写
#                 for y_label in y_labels:
#                     branches[col, y_label] += 1
                    
            IG = Information_Gain(0, branches)
            if IG > IG_max:
                IG_max = IG
                self.dim_split = i
                self.feature_uniq_split = unique_vals.tolist()
                
                
#         print(self.dim_split)
        # Given the result we get by comparing IG, split and generate child node
        i_dim = features[:, self.dim_split]
        
        for attr_val in self.feature_uniq_split:
            indexes = np.where(i_dim == attr_val)
            x = i_dim[indexes]
            
            # new features should remove the split attribute
            new_features = np.delete(self.features, self.dim_split, axis=1)[indexes]       
            new_labels = labels[indexes]
            
            child = TreeNode(new_features.tolist(), new_labels.tolist(), np.unique(new_labels).size)
            
            # set terminate condition
            if new_features.size == 0:
                child.splittable = False
                child.features = None
                # init中已经验证过
#             if np.unique(new_labels).size < 2:
#                 child.splittable = False
            
            self.children.append(child)
        
        # recursion
        for child in self.children:
            if child.splittable:
                child.split()
        
        return
    
    # TODO: predict the branch or the class
    def predict(self, feature):
        # feature: List[any]
        # return: int
        # raise NotImplementedError
        if self.splittable:
#             print(self.dim_split, feature[self.dim_split])
#             print(self.feature_uniq_split)
            idx_child = self.feature_uniq_split.index(feature[self.dim_split])
            feature = np.append(feature[:self.dim_split], feature[self.dim_split + 1:])
#             feature = feature[:self.dim_split] + feature[self.dim_split + 1:]
            return self.children[idx_child].predict(feature)
        else:
#             print(self.cls_max)
            return self.cls_max


In [105]:
def sample_branch_data():
    branch = [[0, 4, 2], [2, 0, 4]]
    return branch


def sample_decision_tree_data():
    features = [['a', 'b'], ['b', 'a'], ['b', 'c'], ['c', 'b']]
    labels = [0, 0, 1, 1]
    return features, labels


def sample_decision_tree_test():
    features = [['a', 'b'], ['b', 'a'], ['b', 'c']]
    labels = [0, 0, 1]
    return features, labels


def sample_decision_tree_pruning():
    features = [[0, 0, 0, 0],
                [0, 0, 0, 1],
                [1, 0, 0, 0],
                [2, 1, 0, 0],
                [2, 2, 1, 0],
                [2, 2, 1, 1],
                [1, 2, 1, 1],
                [0, 1, 0, 0],
                [0, 2, 1, 0],
                [2, 1, 1, 0],
                [0, 1, 1, 1],
                [1, 1, 0, 1],
                [1, 0, 1, 0],
                [2, 1, 0, 1]
                ]
    labels = [0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0]
    validation = [[1, 0, 1, 1],
                  [1, 2, 0, 0],
                  [0, 1, 0, 1],
                  [0, 2, 0, 0],
                  [0, 1, 0, 0],
                  [0, 1, 0, 1],
                  [0, 1, 1, 1],
                  [2, 0, 0, 1],
                  [2, 1, 1, 1],
                  [2, 1, 0, 1],
                  [2, 1, 0, 0]]
    v_labels = [1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1]
    return features, labels, validation, v_labels


def load_decision_tree_data():
    f = open('car.data', 'r')
    white = [[int(num) for num in line.split(',')] for line in f]
    white = np.asarray(white)

    [N, d] = white.shape

    ntr = int(np.round(N * 0.66))
    ntest = N - ntr

    Xtrain = white[:ntr].T[:-1].T
    ytrain = white[:ntr].T[-1].T
    Xtest = white[-ntest:].T[:-1].T
    ytest = white[-ntest:].T[-1].T

    return Xtrain, ytrain, Xtest, ytest


def most_common(lst):
    return max(set(lst), key=lst.count)

In [124]:
def decision_tree_test():
    features, labels = sample_decision_tree_data()

    # build the tree
    dTree = DecisionTree()

    dTree.train(features, labels)

    # print
#     print('Your decision tree: ')
#     print_tree(dTree)
#     print('My decision tree: ')
#     print('branch 0{\n\tdeep: 0\n\tnum of samples for each class: 2 : 2 \n\tsplit by dim 0\n\tbranch 0->0{\n\t\tdeep: '
#           '1\n\t\tnum of samples for each class: 1 \n\t\tclass:0\n\t}\n\tbranch 0->1{\n\t\tdeep: 1\n\t\tnum of '
#           'samples for each class: 1 : 1 \n\t\tsplit by dim 0\n\t\tbranch 0->1->0{\n\t\t\tdeep: 2\n\t\t\tnum of '
#           'samples for each class: 1 \n\t\t\tclass:0\n\t\t}\n\t\tbranch 0->1->1{\n\t\t\tdeep: 2\n\t\t\tnum of '
#           'samples for each class: 1 \n\t\t\tclass:1\n\t\t}\n\t}\n\tbranch 0->2{\n\t\tdeep: 1\n\t\tnum of '
#           'samples for each class: 1 \n\t\tclass:1\n\t}\n}')

    # data
    X_test, y_test = sample_decision_tree_test()

    # testing
    y_est_test = dTree.predict(X_test)
    print('Your estimate test: ', y_est_test)
    print('My estimate test: ', [0, 0, 1])

    
# Train
# features = [['a', 'b'], ['b', 'a'], ['b', 'c'], ['c', 'b']]
# labels = [0, 0, 1, 1] 
# Test
# features = [['a', 'b'], ['b', 'a'], ['b', 'c']]
# labels = [0, 0, 1]
decision_tree_test()

Your estimate test:  [0, 0, 1]
My estimate test:  [0, 0, 1]


In [63]:
A = np.zeros((3,2))
np.count_nonzero(A)

0

In [114]:
print('My decision tree: ')
print('branch 0{\n\tdeep: 0\n\tnum of samples for each class: 2 : 2 \n\tsplit by dim 0\n\tbranch 0->0{\n\t\tdeep: '
      '1\n\t\tnum of samples for each class: 1 \n\t\tclass:0\n\t}\n\tbranch 0->1{\n\t\tdeep: 1\n\t\tnum of '
      'samples for each class: 1 : 1 \n\t\tsplit by dim 0\n\t\tbranch 0->1->0{\n\t\t\tdeep: 2\n\t\t\tnum of '
      'samples for each class: 1 \n\t\t\tclass:0\n\t\t}\n\t\tbranch 0->1->1{\n\t\t\tdeep: 2\n\t\t\tnum of '
      'samples for each class: 1 \n\t\t\tclass:1\n\t\t}\n\t}\n\tbranch 0->2{\n\t\tdeep: 1\n\t\tnum of '
      'samples for each class: 1 \n\t\tclass:1\n\t}\n}')

My decision tree: 
branch 0{
	deep: 0
	num of samples for each class: 2 : 2 
	split by dim 0
	branch 0->0{
		deep: 1
		num of samples for each class: 1 
		class:0
	}
	branch 0->1{
		deep: 1
		num of samples for each class: 1 : 1 
		split by dim 0
		branch 0->1->0{
			deep: 2
			num of samples for each class: 1 
			class:0
		}
		branch 0->1->1{
			deep: 2
			num of samples for each class: 1 
			class:1
		}
	}
	branch 0->2{
		deep: 1
		num of samples for each class: 1 
		class:1
	}
}


In [231]:
def pruning_decision_tree_test():
    # load data
    X_train, y_train, X_test, y_test = sample_decision_tree_pruning()

    # build the tree
    dTree = DecisionTree()
    dTree.train(X_train, y_train)

    # print
    print('Your decision tree:')
    print_tree(dTree)
    print('My decision tree:')
    print('branch 0{\n\tdeep: 0\n\tnum of samples for each class: 5 : 9 \n\tsplit by dim 0\n\tbranch 0->0{\n\t\tdeep: 1'
          '\n\t\tnum of samples for each class: 3 : 2 \n\t\tsplit by dim 1\n\t\tbranch 0->0->0{\n\t\t\tdeep: 2\n\t\t\t'
          'num of samples for each class: 3 \n\t\t\tclass:0\n\t\t}\n\t\tbranch 0->0->1{\n\t\t\tdeep: 2\n\t\t\tnum of '
          'samples for each class: 2 \n\t\t\tclass:1\n\t\t}\n\t}\n\tbranch 0->1{\n\t\tdeep: 1\n\t\tnum of samples for '
          'each class: 4 \n\t\tclass:1\n\t}\n\tbranch 0->2{\n\t\tdeep: 1\n\t\tnum of samples for each class: 2 : 3 '
          '\n\t\tsplit by dim 2\n\t\tbranch 0->2->0{\n\t\t\tdeep: 2\n\t\t\tnum of samples for each class: 3 \n\t\t\t'
          'class:1\n\t\t}\n\t\tbranch 0->2->1{\n\t\t\tdeep: 2\n\t\t\tnum of samples for each class: 2 \n\t\t\tclass:0'
          '\n\t\t}\n\t}\n}')
    
    y_est_test = dTree.predict(X_test)
    print('Your estimate test: ', y_est_test)
    
    reduced_error_prunning(dTree, X_test, y_test)

    print('Your decision tree after pruning:')
    print_tree(dTree)
    print('My decision tree after pruning:')
    print('branch 0{\n\tdeep: 0\n\tnum of samples for each class: 5 : 9 \n\tsplit by dim 0\n\tbranch 0->0{\n\t\tdeep: '
          '1\n\t\tnum of samples for each class: 3 : 2 \n\t\tsplit by dim 1\n\t\tbranch 0->0->0{\n\t\t\tdeep: 2\n\t\t\t'
          'num of samples for each class: 3 \n\t\t\tclass:0\n\t\t}\n\t\tbranch 0->0->1{\n\t\t\tdeep: 2\n\t\t\tnum of '
          'samples for each class: 2 \n\t\t\tclass:1\n\t\t}\n\t}\n\tbranch 0->1{\n\t\tdeep: 1\n\t\tnum of samples for '
          'each class: 4 \n\t\tclass:1\n\t}\n\tbranch 0->2{\n\t\tdeep: 1\n\t\tnum of samples for each class: 2 : 3 '
          '\n\t\tclass:1\n\t}\n}')
    
pruning_decision_tree_test()

Your decision tree:
branch 0{
	deep: 0
	num of samples for each class: 5 : 9 
	split by dim 0
	branch 0->0{
		deep: 1
		num of samples for each class: 3 : 2 
		split by dim 1
		branch 0->0->0{
			deep: 2
			num of samples for each class: 3 
			class: 0
		}
		branch 0->0->1{
			deep: 2
			num of samples for each class: 2 
			class: 1
		}
	}
	branch 0->1{
		deep: 1
		num of samples for each class: 4 
		class: 1
	}
	branch 0->2{
		deep: 1
		num of samples for each class: 2 : 3 
		split by dim 2
		branch 0->2->0{
			deep: 2
			num of samples for each class: 3 
			class: 1
		}
		branch 0->2->1{
			deep: 2
			num of samples for each class: 2 
			class: 0
		}
	}
}
My decision tree:
branch 0{
	deep: 0
	num of samples for each class: 5 : 9 
	split by dim 0
	branch 0->0{
		deep: 1
		num of samples for each class: 3 : 2 
		split by dim 1
		branch 0->0->0{
			deep: 2
			num of samples for each class: 3 
			class:0
		}
		branch 0->0->1{
			deep: 2
			num of samples for each class: 2 
			class:1
		}

In [232]:
import numpy as np
import pandas as pd

data = pd.read_csv('car.data.csv', low_memory=False, sep=',', na_values='?', header=None).values

N = data.shape[0]

# np.random.shuffle(data)
# prepare data

ntr = int(np.round(N * 0.8))
nval = int(np.round(N * 0.15))
ntest = N - ntr - nval

# spliting training, validation, and test
x_train = data[:ntr].T[:-1].T
y_train = data[:ntr].T[-1].T

x_val = data[ntr:ntr + nval].T[:-1].T
y_val = data[ntr:ntr + nval].T[-1].T

x_test = data[-ntest:].T[:-1].T
y_test = data[-ntest:].T[-1].T


dTree = DecisionTree()
dTree.train(x_train, y_train.tolist())

print('Your decision tree: ')
print_tree(dTree)

y_est_test = dTree.predict(x_test)
print('Your estimate test: ', y_est_test)
reduced_error_prunning(dTree, x_val, y_val.tolist())

y_est_test = dTree.predict(x_test)
print('Your estimate test: ', y_est_test)

Your decision tree: 
branch 0{
	deep: 0
	num of samples for each class: 987 : 306 : 23 : 26 
	split by dim 5
	branch 0->0{
		deep: 1
		num of samples for each class: 448 
		class: 0
	}
	branch 0->1{
		deep: 1
		num of samples for each class: 307 : 127 : 13 
		split by dim 3
		branch 0->1->0{
			deep: 2
			num of samples for each class: 150 
			class: 0
		}
		branch 0->1->1{
			deep: 2
			num of samples for each class: 82 : 62 : 6 
			split by dim 3
			branch 0->1->1->0{
				deep: 3
				num of samples for each class: 42 : 8 
				split by dim 0
				branch 0->1->1->0->0{
					deep: 4
					num of samples for each class: 18 
					class: 0
				}
				branch 0->1->1->0->1{
					deep: 4
					num of samples for each class: 8 : 8 
					split by dim 0
					branch 0->1->1->0->1->0{
						deep: 5
						num of samples for each class: 4 : 4 
						split by dim 0
						branch 0->1->1->0->1->0->0{
							deep: 6
							num of samples for each class: 1 : 1 
							class: 0
						}
						branch 0->1->1->0

Your estimate test:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0]


In [221]:
A = np.array([1,2,3])


array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [235]:
B = np.array([[1], [2]])
np.delete(B, 0, axis=1).tolist()

[[], []]